In [6]:

# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Visualización
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluar linealidad de las relaciones entre las variables
# y la distribución de las variables
# ------------------------------------------------------------------------------
import scipy.stats as stats

from scipy.stats import shapiro, levene, mannwhitneyu
from scipy.stats import ttest_ind, norm, chi2_contingency

# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

# Gestión de los warnings
# -----------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

Ejercicio
•	Fase 1: Exploración y Limpieza
1.	Exploración Inicial:
	Realiza una exploración inicial de los datos para identificar posibles problemas, como valores nulos, atípicos o datos faltantes en las columnas relevantes.
	Utiliza funciones de Pandas para obtener información sobre la estructura de los datos, la presencia de valores nulos y estadísticas básicas de las columnas involucradas.
	Une los dos conjuntos de datos de la forma más eficiente.

In [7]:
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

In [8]:
# cargamos todos los conjuntos de datos: En principio los he cargado de esta manera pero considero que es mejor utilizar esta columna indice para realizar la unión de los datos.
dfa = pd.read_csv("Customer Flight Activity.csv", index_col = 0)



In [9]:
dlh = pd.read_csv("Customer Loyalty History.csv", index_col = 0)

In [10]:
# Cargamos dfa con indice Loyalty Number como numero de identificación del cliente, visualizamos las primeras filas:
dfa = pd.read_csv("Customer Flight Activity.csv", index_col = "Loyalty Number")
dfa.head(2)


,Year,Month,Flights Booked,Flights with Companions,Total Flights,Distance,Points Accumulated,Points Redeemed,Dollar Cost Points Redeemed
Loyalty Number,,,,,,,,,
100018,2017,1,3,0,3,1521,152.0,0,0
100102,2017,1,10,4,14,2030,203.0,0,0


In [11]:
# Cargamos dlh con indice Loyalty Number como numero de identificación del cliente, visualizamos las primeras filas:
dlh = pd.read_csv("Customer Loyalty History.csv", index_col = "Loyalty Number")
dlh.head(2)

,Country,Province,City,Postal Code,Gender,Education,Salary,Marital Status,Loyalty Card,CLV,Enrollment Type,Enrollment Year,Enrollment Month,Cancellation Year,Cancellation Month
Loyalty Number,,,,,,,,,,,,,,,
480934,Canada,Ontario,Toronto,M2Z 4K1,Female,Bachelor,83236.0,Married,Star,3839.14,Standard,2016,2,NaN,NaN
549612,Canada,Alberta,Edmonton,T3G 6Y6,Male,College,NaN,Divorced,Star,3839.61,Standard,2016,3,NaN,NaN


Revisamos el numero de filas y columnas para cada conjunto de datos y el detalle de sus columnas

In [33]:
dfa.shape

(405624, 9)

In [34]:
dlh.shape

(16737, 15)

In [35]:
dlh.columns

Index(['Country', 'Province', 'City', 'Postal Code', 'Gender', 'Education',
       'Salary', 'Marital Status', 'Loyalty Card', 'CLV', 'Enrollment Type',
       'Enrollment Year', 'Enrollment Month', 'Cancellation Year',
       'Cancellation Month'],
      dtype='object')

In [36]:
dfa.columns

Index(['Year', 'Month', 'Flights Booked', 'Flights with Companions',
       'Total Flights', 'Distance', 'Points Accumulated', 'Points Redeemed',
       'Dollar Cost Points Redeemed'],
      dtype='object')

In [12]:
# Fusionar los DataFrames mediante el índice 'Loyalty Number'
df= pd.merge(dlh, dfa, left_index=True, right_index=True)
df.columns

Index(['Country', 'Province', 'City', 'Postal Code', 'Gender', 'Education',
       'Salary', 'Marital Status', 'Loyalty Card', 'CLV', 'Enrollment Type',
       'Enrollment Year', 'Enrollment Month', 'Cancellation Year',
       'Cancellation Month', 'Year', 'Month', 'Flights Booked',
       'Flights with Companions', 'Total Flights', 'Distance',
       'Points Accumulated', 'Points Redeemed', 'Dollar Cost Points Redeemed'],
      dtype='object')

In [64]:
# realizamos un analisis inicial de todo el dataframe
df.head(3)


,country,province,city,postal_code,gender,education,salary,marital_status,loyalty_card,clv,enrollment_type,enrollment_year,enrollment_month,cancellation_year,cancellation_month,year,month,flights_booked,flights_with_companions,total_flights,distance,points_accumulated,points_redeemed,dollar_cost_points_redeemed
Loyalty Number,,,,,,,,,,,,,,,,,,,,,,,,
100018,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.2,Standard,2016,8,NaN,NaN,2017,1,3,0,3,1521,152.0,0,0
100018,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.2,Standard,2016,8,NaN,NaN,2017,2,2,2,4,1320,132.0,0,0
100018,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.2,Standard,2016,8,NaN,NaN,2018,10,6,4,10,3110,311.0,385,31


In [74]:
# una vez que tengamos nuestro DataFrame preparado con todas las columnas que queremos vamos a crear una función que no haga una exploración inicial del conjunto de datos
def eda(dataframe):
    """
    Realiza un análisis exploratorio básico de un DataFrame, mostrando información sobre duplicados,
    valores nulos, tipos de datos, valores únicos para columnas categóricas y estadísticas descriptivas
    para columnas categóricas y numéricas

    Parámetros:
    - dataframe (DataFrame): El DataFrame que se va a explorar.
    
    Returns: 
    No devuelve nada directamente, pero imprime en la consola la información exploratoria.
    """
    
    print(f"Los duplicados que tenemos en el conjunto de datos son: {dataframe.duplicated().sum()}")
    print("\n ..................... \n")
    
    print(f"el numero de filas y columnas del df son : {df.shape}")
    print("\n ..................... \n")
    
    # generamos un DataFrame para los valores nulos
    print("Los nulos que tenemos en el conjunto de datos son:")
    df_nulos = pd.DataFrame(dataframe.isnull().sum() / dataframe.shape[0] * 100, columns = ["%_nulos"])
    display(df_nulos[df_nulos["%_nulos"] > 0])
    
    # generamos un DataFrame para tipos de datos
    print("\n ..................... \n")
    print(f"Los tipos de las columnas son:")
    display(pd.DataFrame(dataframe.dtypes, columns = ["tipo_dato"]))

    
    # principales estadisticos de todas las columnas
    print("\n ..................... \n")

    # estadisticos columnas numericas
    print(f"estadisticos columnas numericas")
    display(df.describe().T)
    print("\n ..................... \n")


    # estadisticos columnas categoricas
    print(f"estadisticos columnas categoricas")
    display(df.describe(include = "O").T)
    
    
    print("\n ..................... \n")
    print("Los valores que tenemos para las columnas categóricas son: ")
    dataframe_categoricas = dataframe.select_dtypes(include = "O")
    
    for col in dataframe_categoricas.columns:
        print(f"La columna {col.upper()} tiene las siguientes valore únicos:")
        display(pd.DataFrame(dataframe[col].value_counts()).head())    
    


In [75]:
eda(df)

Los duplicados que tenemos en el conjunto de datos son: 1864

 ..................... 

el numero de filas y columnas del df son : (405624, 23)

 ..................... 

Los nulos que tenemos en el conjunto de datos son:


,%_nulos
salary,25.312112
cancellation_year,87.657535
cancellation_month,87.657535



 ..................... 

Los tipos de las columnas son:


,tipo_dato
province,object
city,object
postal_code,object
gender,object
education,object
salary,float64
marital_status,object
loyalty_card,object
clv,float64
enrollment_type,object



 ..................... 

estadisticos columnas numericas


...

,marital_status
married,235800
single,108648
divorced,61176


La columna LOYALTY_CARD tiene las siguientes valore únicos:


,loyalty_card
star,184224
nova,137568
aurora,83832


La columna ENROLLMENT_TYPE tiene las siguientes valore únicos:


,enrollment_type
standard,382200
2018_promotion,23424


In [124]:
df.columns

Index(['Country', 'Province', 'City', 'Postal Code', 'Gender', 'Education',
       'Salary', 'Marital Status', 'Loyalty Card', 'CLV', 'Enrollment Type',
       'Enrollment Year', 'Enrollment Month', 'Cancellation Year',
       'Cancellation Month', 'Year', 'Month', 'Flights Booked',
       'Flights with Companions', 'Total Flights', 'Distance',
       'Points Accumulated', 'Points Redeemed', 'Dollar Cost Points Redeemed'],
      dtype='object')

In [13]:
df_sorted = df.sort_values(by='Salary')


df_sorted['Salary']


Loyalty Number
152016   -58486.0
152016   -58486.0
152016   -58486.0
152016   -58486.0
152016   -58486.0
           ...   
999982        NaN
999982        NaN
999982        NaN
999982        NaN
999982        NaN
Name: Salary, Length: 405624, dtype: float64

In [14]:
df['Salary']

Loyalty Number
100018    92552.0
100018    92552.0
100018    92552.0
100018    92552.0
100018    92552.0
           ...   
999986    46594.0
999986    46594.0
999986    46594.0
999986    46594.0
999986    46594.0
Name: Salary, Length: 405624, dtype: float64

In [152]:
df.shape[0]

405624

En principio la columna country es el mismo valor para todos por lo que no considero necesaria esa columna
Si consideramos que los datos nulos se consideran altos a partir de un  5-10%. En la columna salary tenemos un 11.8% de valores negativos y 25.31% de nulos. Entiendo que esta columna es importante para el analisis por la que revisaremos con mas detalle para determinar que hacer con estos negativos.Las columnas
cancellation_year y cancellation_month presentan "ambas" un 87.65% de nulos. La columna year años 2017/2018

# Cleaning

In [15]:
# vamos a cambiar las columnas, utilizando una listc. itero for col in df.columns: Itera sobre cada columna en el DataFrame df.

# col.lower(): Convierte el nombre de la columna a minúsculas.
# .replace(" ", "_"): Reemplaza cualquier espacio en blanco en el nombre de la columna con un guion bajo "_".
columnas = [col.lower().replace(" ", "_") for col in df.columns]
df.columns = columnas
df.head(2)

,country,province,city,postal_code,gender,education,salary,marital_status,loyalty_card,clv,enrollment_type,enrollment_year,enrollment_month,cancellation_year,cancellation_month,year,month,flights_booked,flights_with_companions,total_flights,distance,points_accumulated,points_redeemed,dollar_cost_points_redeemed
Loyalty Number,,,,,,,,,,,,,,,,,,,,,,,,
100018,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.2,Standard,2016,8,NaN,NaN,2017,1,3,0,3,1521,152.0,0,0
100018,Canada,Alberta,Edmonton,T9G 1W3,Female,Bachelor,92552.0,Married,Aurora,7919.2,Standard,2016,8,NaN,NaN,2017,2,2,2,4,1320,132.0,0,0


In [16]:
def limpiar_columna(columna):
    """
    Convierte los valores de una columna a minúsculas y reemplaza los espacios con '_'.
    
    Parámetros:
    - columna: La columna que se desea limpiar.
    
    Retorna:
    - La columna limpia.
    """
    # Convertir a minúsculas y reemplazar espacios con '_'
    columna = columna.str.lower().str.replace(' ', '_')
    return columna



In [17]:
# Lista de columnas que deseas limpiar
columnas_por_cambiar = ['province', 'city', 'postal_code', 'gender', 'education', 'marital_status', 'loyalty_card', "enrollment_type"]



In [18]:
# Aplicar la función a las columnas seleccionadas
df[columnas_por_cambiar] = df[columnas_por_cambiar].apply(limpiar_columna)

In [62]:
df.head(1)

,country,province,city,postal_code,gender,education,salary,marital_status,loyalty_card,clv,enrollment_type,enrollment_year,enrollment_month,cancellation_year,cancellation_month,year,month,flights_booked,flights_with_companions,total_flights,distance,points_accumulated,points_redeemed,dollar_cost_points_redeemed
Loyalty Number,,,,,,,,,,,,,,,,,,,,,,,,
100018,Canada,alberta,edmonton,t9g_1w3,female,bachelor,92552.0,married,aurora,7919.2,standard,2016,8,NaN,NaN,2017,1,3,0,3,1521,152.0,0,0.0


In [19]:
df['dollar_cost_points_redeemed'] = df['dollar_cost_points_redeemed'].astype(float)

In [20]:
#  la columna salarios tiene valores negativos por lo que quiero saber que porcentaje de negativos tiene ya que ademas tiene un alto numero de nulos
negatives_count = df[df['salary'] < 0]['salary'].count()
porc_negatives_count = negatives_count/df.shape[0]*100
porc_negatives_count

0.11833619312466717

In [102]:
negatives_count = df[df['salary'] < 0]['salary'].count()
negatives_count

480

In [103]:
negatives_counts2 = df[df['salary'] < 0]['salary'].value_counts()
negatives_counts2 

-10605.0    24
-58486.0    24
-57297.0    24
-34079.0    24
-46303.0    24
-19332.0    24
-19325.0    24
-49830.0    24
-45962.0    24
-43234.0    24
-46470.0    24
-17534.0    24
-9081.0     24
-26322.0    24
-39503.0    24
-47310.0    24
-46683.0    24
-49001.0    24
-31911.0    24
-12497.0    24
Name: salary, dtype: int64

In [21]:
# consideraré que el salario se ha introducido por error el salario negativo asi que lo convierto a positivo(esto lo haré en un principio)
df["salary"] = abs(df["salary"])

In [72]:
df["salary"].describe()

count    302952.000000
mean      79381.440228
std       34735.904533
min        9081.000000
25%       59262.000000
50%       73479.000000
75%       88612.000000
max      407228.000000
Name: salary, dtype: float64

In [77]:
df.columns

Index(['province', 'city', 'postal_code', 'gender', 'education', 'salary',
       'marital_status', 'loyalty_card', 'clv', 'enrollment_type',
       'enrollment_year', 'enrollment_month', 'cancellation_year',
       'cancellation_month', 'year', 'month', 'flights_booked',
       'flights_with_companions', 'total_flights', 'distance',
       'points_accumulated', 'points_redeemed', 'dollar_cost_points_redeemed'],
      dtype='object')

In [22]:
# He borrado la columna pais porque corresponde al mismo pais, información que puede suministrarse en el enunciado
df.drop(['country'], axis=1, inplace=True)

In [23]:
# borramos duplicados son el 0.46% de los datos 
df.drop_duplicates(inplace=True)

In [81]:
df.shape

(403760, 23)

Fase 2: Visualización
Usando las herramientas de visualización que has aprendido durante este módulo, contesta a las siguientes gráficas usando la mejor gráfica que consideres:

2. ¿Existe una relación entre la distancia de los vuelos y los puntos acumulados por los clientes?
3. ¿Cuál es la distribución de los clientes por provincia o estado?
4. ¿Cómo se compara el salario promedio entre los diferentes niveles educativos de los clientes?
5. ¿Cuál es la proporción de clientes con diferentes tipos de tarjetas de fidelidad?
6. ¿Cómo se distribuyen los clientes según su estado civil y género?


1. ¿Cómo se distribuye la cantidad de vuelos reservados por mes durante el año?

In [26]:
# Agrupar los datos por año y mes, y sumar la cantidad de vuelos reservados en cada grupo
vuelos_por_anio_mes = df.groupby(['year', 'month'])['flights_booked'].sum()

# Crear un nuevo DataFrame con los resultados
vuelos_df = vuelos_por_anio_mes.reset_index()
vuelos_df


,year,month,flights_booked
0,2017,1,47766
1,2017,2,47691
2,2017,3,64271
3,2017,4,55720
4,2017,5,65279
5,2017,6,80493
6,2017,7,89288
7,2017,8,78692
8,2017,9,63304
9,2017,10,60351


In [28]:
df.head(1)

,province,city,postal_code,gender,education,salary,marital_status,loyalty_card,clv,enrollment_type,enrollment_year,enrollment_month,cancellation_year,cancellation_month,year,month,flights_booked,flights_with_companions,total_flights,distance,points_accumulated,points_redeemed,dollar_cost_points_redeemed
Loyalty Number,,,,,,,,,,,,,,,,,,,,,,,
100018,alberta,edmonton,t9g_1w3,female,bachelor,92552.0,married,aurora,7919.2,standard,2016,8,NaN,NaN,2017,1,3,0,3,1521,152.0,0,0.0


In [29]:
# Filtrar el DataFrame original para el año 2017
df_2017 = df[df['year'] == 2017]

# Calcular la suma de vuelos reservados por mes para el año 2017
vuelos_por_mes_2017 = df_2017.groupby('month')['flights_booked'].sum()

# Crear un nuevo DataFrame para el año 2017
df_2017_resultado = vuelos_por_mes_2017.reset_index()
print("\nDataFrame para el año 2018:")
df_2017_resultado


DataFrame para el año 2018:


,month,flights_booked
0,1,47766
1,2,47691
2,3,64271
3,4,55720
4,5,65279
5,6,80493
6,7,89288
7,8,78692
8,9,63304
9,10,60351


In [30]:


# Filtrar el DataFrame original para el año 2018
df_2018 = df[df['year'] == 2018]

# Calcular la suma de vuelos reservados por mes para el año 2018
vuelos_por_mes_2018 = df_2018.groupby('month')['flights_booked'].sum()

# Crear un nuevo DataFrame para el año 2018
df_2018_resultado = vuelos_por_mes_2018.reset_index()

# Mostrar los resultados


print("\nDataFrame para el año 2018:")
df_2018_resultado




DataFrame para el año 2018:


,month,flights_booked
0,1,49170
1,2,48643
2,3,66928
3,4,56791
4,5,76891
5,6,92749
6,7,104143
7,8,90512
8,9,73075
9,10,69110


In [ ]:

# Datos para el primer gráfico
meses_1 = [1, 2, 3, 4]
vuelos_reservados_1 = [47766, 47691, 64271, 55720]

# Datos para el segundo gráfico

meses_2 = [1, 2, 3, 4]
vuelos_reservados_2 = [48000, 47000, 65000, 56000]

# Crear el subplot
fig, axs = plt.subplots(1, 2, figsize=(12, 6))

# Primer gráfico
sns.barplot(x="month", y="flights_booked", color='skyblue', ax=axs[0])
axs[0].set_title('Gráfico 1')
axs[0].set_xlabel('Mes')
axs[0].set_ylabel('Número de vuelos reservados')

# Segundo gráfico
sns.barplot(x="month", y="flights_booked", color='salmon', ax=axs[1])
axs[1].set_title('Gráfico 2')
axs[1].set_xlabel('Mes')
axs[1].set_ylabel('Número de vuelos reservados')

# Ajustar espaciado
plt.tight_layout()

# Mostrar el subplot
plt.show()


In [ ]:
sns.histplot(x = 'month', data = df)
plt.xlabel('Tasa natalidad')
plt.title('Distribución natalidad');

In [ ]:
df_2017.info()

NameError: name 'df_2017' is not defined

In [ ]:
sns.histplot(x = 'fertility_rate', data = df)
plt.xlabel('Tasa natalidad')
plt.title('Distribución natalidad');

In [86]:

# convertimos la columna de fecha a formato datetime["static_fire_date_utc"] = pd.to_datetime(df["static_fire_date_utc"])
# convertimos la columna de fecha a formato datetime
vuelos_df["year"] = pd.to_datetime(vuelos_df["year"])


In [ ]:
df[['year', 'month']] = df['static_fire_date_utc'].apply(lambda x: pd.Series([x.year, x.month]))
df.head()

In [ ]:
# Agrupamos los datos por mes y sumar la cantidad de vuelos reservados
vuelos_por_mes = df.groupby('Month')['Flights Booked'].sum()

In [31]:
# una vez que tengamos nuestro DataFrame preparado con todas las columnas que queremos vamos a crear una función que no haga una exploración inicial del conjunto de datos
def exploracion_dataframe(dataframe, columna_control):
    """
    Realiza un análisis exploratorio básico de un DataFrame, mostrando información sobre duplicados,
    valores nulos, tipos de datos, valores únicos para columnas categóricas y estadísticas descriptivas
    para columnas categóricas y numéricas, agrupadas por la columna de control.

    Parámetros:
    - dataframe (DataFrame): El DataFrame que se va a explorar.
    - columna_control (str): El nombre de la columna que se utilizará como control para dividir el DataFrame.

    Returns: 
    No devuelve nada directamente, pero imprime en la consola la información exploratoria.
    """
    
    print(f"Los duplicados que tenemos en el conjunto de datos son: {dataframe.duplicated().sum()}")
    print("\n ..................... \n")
    
    
    # generamos un DataFrame para los valores nulos
    print("Los nulos que tenemos en el conjunto de datos son:")
    df_nulos = pd.DataFrame(dataframe.isnull().sum() / dataframe.shape[0] * 100, columns = ["%_nulos"])
    display(df_nulos[df_nulos["%_nulos"] > 0])
    
    print("\n ..................... \n")
    print(f"Los tipos de las columnas son:")
    display(pd.DataFrame(dataframe.dtypes, columns = ["tipo_dato"]))
    
    
    print("\n ..................... \n")
    print("Los valores que tenemos para las columnas categóricas son: ")
    dataframe_categoricas = dataframe.select_dtypes(include = "O")
    
    for col in dataframe_categoricas.columns:
        print(f"La columna {col.upper()} tiene las siguientes valore únicos:")
        display(pd.DataFrame(dataframe[col].value_counts()).head())    
    
    # como estamos en un problema de A/B testing y lo que realmente nos importa es comparar entre el grupo de control y el de test, los principales estadísticos los vamos a sacar de cada una de las categorías
    
    for categoria in dataframe[columna_control].unique():
        
        dataframe_filtrado = dataframe[dataframe[columna_control] == categoria]
    
        print("\n ..................... \n")
        print(f"Los principales estadísticos de las columnas categóricas para el {categoria.upper()} son: ")
        display(dataframe_filtrado.describe(include = "O").T)
        
        print("\n ..................... \n")
        print(f"Los principales estadísticos de las columnas numéricas para el {categoria.upper()} son: ")
        display(dataframe_filtrado.describe().T)

In [49]:
df['loyalty_card'].unique()

array(['Aurora', 'Nova', 'Star'], dtype=object)

In [40]:
exploracion_dataframe(df,"education")

Los duplicados que tenemos en el conjunto de datos son: 1864

 ..................... 

Los nulos que tenemos en el conjunto de datos son:


,%_nulos
salary,25.312112
cancellation_year,87.657535
cancellation_month,87.657535



 ..................... 

Los tipos de las columnas son:


,tipo_dato
country,object
province,object
city,object
postal_code,object
gender,object
education,object
salary,float64
marital_status,object
loyalty_card,object
clv,float64



 ..................... 

Los valores que tenemos para las columnas categóricas son: 
La columna COUNTRY tiene las siguientes valore únicos:


...

,count,mean,std,min,25%,50%,75%,max
salary,19008.0,60986.544192,12723.916043,-49830.00,52044.75,61806.00,71344.00,93875.0
clv,19008.0,7722.940328,6657.946709,2272.34,3707.28,5269.28,8991.98,43290.5
enrollment_year,19008.0,2015.193182,1.984666,2012.00,2013.00,2015.00,2017.00,2018.0
enrollment_month,19008.0,6.609848,3.457341,1.00,4.00,7.00,10.00,12.0
cancellation_year,2256.0,2016.276596,1.394860,2013.00,2015.00,2016.50,2017.00,2018.0
cancellation_month,2256.0,6.574468,3.738188,1.00,3.00,7.00,10.00,12.0
year,19008.0,2017.500000,0.500013,2017.00,2017.00,2017.50,2018.00,2018.0
month,19008.0,6.500000,3.452143,1.00,3.75,6.50,9.25,12.0
flights_booked,19008.0,4.155777,5.234551,0.00,0.00,1.00,8.00,21.0
flights_with_companions,19008.0,1.043718,2.087937,0.00,0.00,0.00,1.00,11.0



 ..................... 

Los principales estadísticos de las columnas categóricas para el DOCTOR son: 


,count,unique,top,freq
country,17856,1,Canada,17856
province,17856,11,Ontario,5640
city,17856,29,Toronto,3312
postal_code,17856,55,V6E 3Z3,888
gender,17856,2,Female,9024
education,17856,1,Doctor,17856
marital_status,17856,3,Married,11256
loyalty_card,17856,3,Star,8448
enrollment_type,17856,2,Standard,17088



 ..................... 

Los principales estadísticos de las columnas numéricas para el DOCTOR son: 


,count,mean,std,min,25%,50%,75%,max
salary,17856.0,178335.439516,70307.642502,48109.00,124343.7500,180440.00,235069.250,407228.00
clv,17856.0,7815.538978,6466.157567,2119.89,4056.4125,5619.19,8580.735,58207.13
enrollment_year,17856.0,2015.072581,1.949028,2012.00,2013.0000,2015.00,2017.000,2018.00
enrollment_month,17856.0,6.896505,3.389085,1.00,4.0000,7.00,10.000,12.00
cancellation_year,2328.0,2016.371134,1.394837,2013.00,2015.0000,2016.00,2018.000,2018.00
cancellation_month,2328.0,7.000000,3.446940,1.00,4.0000,7.00,10.000,12.00
year,17856.0,2017.500000,0.500014,2017.00,2017.0000,2017.50,2018.000,2018.00
month,17856.0,6.500000,3.452149,1.00,3.7500,6.50,9.250,12.00
flights_booked,17856.0,4.146281,5.250093,0.00,0.0000,1.00,8.000,21.00
flights_with_companions,17856.0,1.018873,2.051570,0.00,0.0000,0.00,1.000,11.00


In [ ]:
# Crear subgráficos
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(20, 5))

# Gráfico 1: Tasa de Clicks (CTR)
sns.barplot(x="campaign_name", y="CTR", data=df, ax=axes[0])
axes[0].set_title("Tasa de Clicks")

# Gráfico 2: Tasa de Conversión (CR)
sns.barplot(x="campaign_name", y="CR", data=df, ax=axes[1])
axes[1].set_title("Tasa de Conversión")

# Gráfico 3: Costo por Adquisición (CPA)
sns.barplot(x="campaign_name", y="CPA", data=df, ax=axes[2])
axes[2].set_title("Coste por Adquisición");

In [51]:
# recordemos cuántos nulos tenemos y en que porcentaje están para decidir correctamente que método de imputación usamos:
# solo tenemos nulos en variables numéricas, por lo que solo lo haremos para este tipo de variables
nulos_esta_num = df[df.columns[df.isnull().any()]].select_dtypes(include = np.number).columns
print("Las columnas numéricas que tienen nulos son : \n ")
print(nulos_esta_num)

Las columnas numéricas que tienen nulos son : 
 
Index(['salary', 'cancellation_year', 'cancellation_month'], dtype='object')


In [ ]:
# como en todos los casos tenemos menos de un 5% de nulos, optaremos por reemplazar por la media o la mediana, para elegir la mejor métrica hagamos un 'decribe()' para decidir que estadístico es mejor
df[nulos_esta_num].describe().T.loc[:, ["mean", "50%"]]

Algunos valores difieren entre la media y la mediana, como es el caso de `of_impressions`, `reach`, `of_purchase`, `CTR`, `CR`, `CPA`. En estos casos, lo correcto sería reemplazar por la mediana. En el resto de las columnas, podríamos reeamplazar por la media o la mediana. Para simplificar el problema, reemplazaremos por la mediana  en todas las columnas. 

In [ ]:
# reemplazamos los valores nulos
for columna in nulos_esta_num:
    mediana = df[columna].median()
    df[columna] = df[columna].fillna(mediana)
    
# comprobamos que ya no queden valores nulos
df.isnull().sum()

Realizamos un EDA para cada uno de los conjuntos de datos: